## IMPORTS

In [2]:
import pandas as pd
import numpy as np
import time 
import winsound         # for sound  
import time             # for sleep
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, make_scorer,  classification_report
from sklearn.preprocessing import StandardScaler
import winsound         # for sound  
import time             # for sleep

In [3]:
ini_total = time.time()

## 1 - Carregando train_df resultado da EDA

In [5]:
ini = time.time()
train_df = pd.read_csv("./data/train_data_after_EDA.csv")
fim = time.time()
print ("Tempo de execução em segundos:", fim - ini)
train_df.info()

Tempo de execução em segundos: 0.9063689708709717
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573740 entries, 0 to 573739
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Claim Identifier                573740 non-null  int64  
 1   Accident Date                   570052 non-null  object 
 2   Age at Injury                   568335 non-null  float64
 3   Alternative Dispute Resolution  573740 non-null  int64  
 4   Attorney/Representative         573740 non-null  int64  
 5   Average Weekly Wage             545105 non-null  float64
 6   Birth Year                      571554 non-null  float64
 7   Carrier Name                    573740 non-null  float64
 8   Carrier Type                    573740 non-null  int64  
 9   Claim Injury Type               573740 non-null  int64  
 10  County of Injury                573740 non-null  float64
 11  COVID-19 Indicator          

In [6]:
train_df = train_df.drop(["Accident Date", "Birth Year"], axis=1 )
print("Dropando 'Accident Date' e 'Birth Year', conforme EDA")
train_df.info()

Dropando 'Accident Date' e 'Birth Year', conforme EDA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573740 entries, 0 to 573739
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Claim Identifier                573740 non-null  int64  
 1   Age at Injury                   568335 non-null  float64
 2   Alternative Dispute Resolution  573740 non-null  int64  
 3   Attorney/Representative         573740 non-null  int64  
 4   Average Weekly Wage             545105 non-null  float64
 5   Carrier Name                    573740 non-null  float64
 6   Carrier Type                    573740 non-null  int64  
 7   Claim Injury Type               573740 non-null  int64  
 8   County of Injury                573740 non-null  float64
 9   COVID-19 Indicator              573740 non-null  int64  
 10  District Name                   573740 non-null  float64
 11  Gender                  

In [7]:
#para rodar na marra inicialmente 2, depois tem que fazer as retiradas de nan corretamente
#train_df.replace(np.nan, 0, inplace = True)

In [8]:
X = train_df.drop(["Claim Injury Type"], axis = 1)
y = train_df["Claim Injury Type"]

In [9]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train_ok = X_train.copy()
X_test_ok = X_test.copy()
y_train_ok = y_train.copy()
y_test_ok = y_test.copy()

In [11]:
X_train_ok.info()

<class 'pandas.core.frame.DataFrame'>
Index: 458992 entries, 478483 to 121958
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Claim Identifier                458992 non-null  int64  
 1   Age at Injury                   454629 non-null  float64
 2   Alternative Dispute Resolution  458992 non-null  int64  
 3   Attorney/Representative         458992 non-null  int64  
 4   Average Weekly Wage             435924 non-null  float64
 5   Carrier Name                    458992 non-null  float64
 6   Carrier Type                    458992 non-null  int64  
 7   County of Injury                458992 non-null  float64
 8   COVID-19 Indicator              458992 non-null  int64  
 9   District Name                   458992 non-null  float64
 10  Gender                          458992 non-null  float64
 11  IME-4 Count                     458992 non-null  float64
 12  Industry Code   

In [12]:
print("features com missed values")
missValMean = ['Age at Injury','Average Weekly Wage', 'Years Past Accident','Assembly Years past Accident']
missValMode = ['Industry Code']

features com missed values


In [13]:
#colocando medias nas colunas numericas continuas e moda nas categoricas, para teste e validação separadas (Data Linkage)
X_train_ok[missValMean] = X_train_ok[missValMean].fillna(X_train_ok[missValMean].mean())
print("ok")

ok


In [14]:
X_test_ok[missValMean] = X_test_ok[missValMean].fillna(X_test_ok[missValMean].mean())
print("ok")

ok


In [15]:
vMode = X_train_ok['Industry Code'].mode()[0]
X_train_ok['Industry Code'] = X_train_ok['Industry Code'].fillna(vMode)
print("ok")

ok


In [16]:
vMode = X_test_ok['Industry Code'].mode()[0]
X_test_ok['Industry Code'] = X_test_ok['Industry Code'].fillna(vMode)
print("missed values resolved!")

missed values resolved!


In [17]:
X_train_ok.info()

<class 'pandas.core.frame.DataFrame'>
Index: 458992 entries, 478483 to 121958
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Claim Identifier                458992 non-null  int64  
 1   Age at Injury                   458992 non-null  float64
 2   Alternative Dispute Resolution  458992 non-null  int64  
 3   Attorney/Representative         458992 non-null  int64  
 4   Average Weekly Wage             458992 non-null  float64
 5   Carrier Name                    458992 non-null  float64
 6   Carrier Type                    458992 non-null  int64  
 7   County of Injury                458992 non-null  float64
 8   COVID-19 Indicator              458992 non-null  int64  
 9   District Name                   458992 non-null  float64
 10  Gender                          458992 non-null  float64
 11  IME-4 Count                     458992 non-null  float64
 12  Industry Code   

In [18]:
X_test_ok.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114748 entries, 502001 to 355120
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Claim Identifier                114748 non-null  int64  
 1   Age at Injury                   114748 non-null  float64
 2   Alternative Dispute Resolution  114748 non-null  int64  
 3   Attorney/Representative         114748 non-null  int64  
 4   Average Weekly Wage             114748 non-null  float64
 5   Carrier Name                    114748 non-null  float64
 6   Carrier Type                    114748 non-null  int64  
 7   County of Injury                114748 non-null  float64
 8   COVID-19 Indicator              114748 non-null  int64  
 9   District Name                   114748 non-null  float64
 10  Gender                          114748 non-null  float64
 11  IME-4 Count                     114748 non-null  float64
 12  Industry Code   

In [19]:
# Escalar os dados
ini = time.time()
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ok)
X_test_scaled = scaler.transform(X_test_ok)
fim = time.time()
print ("Tempo de execução em segundos:", fim - ini)

Tempo de execução em segundos: 0.1796877384185791


In [20]:
# Logistic Regression básico com K-Fold
print("K-Fold (LogRegression) -------------")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LogisticRegression(
    multi_class='multinomial', 
    max_iter=1000,
    verbose=1, 
    random_state=42,
    class_weight="balanced"
)

K-Fold (LogRegression) -------------


In [21]:
# Avaliar usando F1-score com K-Fold
ini = time.time()
f1_scorer = make_scorer(f1_score, average='weighted')
f1_scores = cross_val_score(model, X_train_scaled, y_train, cv=kf, scoring=f1_scorer)

print(f"F1-scores por Fold: {f1_scores}")
print(f"F1-score médio: {np.mean(f1_scores):.2f}")
fim = time.time()
print ("Tempo de execução em segundos:", fim - ini)

F1-scores por Fold: [0.62226214 0.61810417 0.61676742 0.62000702 0.61757891]
F1-score médio: 0.62
Tempo de execução em segundos: 247.49180698394775


In [22]:
# Ajustar modelo no conjunto de treino e avaliar no teste
model.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000,
                   multi_class='multinomial', random_state=42, verbose=1)

In [23]:
y_pred = model.predict(X_test_scaled)

In [24]:
y_pred

array([1, 4, 1, ..., 4, 1, 1], dtype=int64)

In [25]:
ini = time.time()
print(f"F1-score no conjunto de teste: {f1_score(y_test, y_pred, average='weighted'):.2f}")
fim = time.time()
print ("Tempo de execução em segundos:", fim - ini)

F1-score no conjunto de teste: 0.62
Tempo de execução em segundos: 0.08878183364868164


In [26]:
# GridSearchCV para otimização de hiperparâmetros
print("GridSearchCV (LogRegression) --------------")
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Parâmetro de regularização
    'penalty': ['l2'],       # Tipo de penalidade (L1: LASSO, L2: Ridge regularization)
    'solver': ['lbfgs']        # Solver compatível com multifuncional
}

GridSearchCV (LogRegression) --------------


In [27]:
ini = time.time()
grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42, solver='lbfgs', class_weight="balanced"),
    param_grid,
    scoring=f1_scorer,
    cv=5
)
fim = time.time()
print ("Tempo de execução em segundos:", fim - ini)

Tempo de execução em segundos: 0.0


In [28]:
ini = time.time()
grid_search.fit(X_train_scaled, y_train)
fim = time.time()
print ("Tempo de execução em segundos:", fim - ini)

Tempo de execução em segundos: 1184.9644877910614


In [29]:
print(f"Melhores hiperparâmetros: {grid_search.best_params_}")
print(f"Melhor F1-score no treino (cross-val): {grid_search.best_score_:.2f}")

Melhores hiperparâmetros: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Melhor F1-score no treino (cross-val): 0.62


In [30]:
print("Escolhendo o melhor modelo -----------")
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)
print(f"F1-score do melhor modelo no conjunto de teste: {f1_score(y_test, y_pred_best, average='weighted'):.2f}")
#winsound.Beep(600, 250)
#time.sleep(0.25)

Escolhendo o melhor modelo -----------
F1-score do melhor modelo no conjunto de teste: 0.62


In [31]:
print("Classification Report -------------------")
report = classification_report(y_test, y_pred_best)
print(report)

Classification Report -------------------
              precision    recall  f1-score   support

           0       0.25      0.67      0.37      2451
           1       0.88      0.86      0.87     58158
           2       0.21      0.23      0.22     13692
           3       0.76      0.27      0.39     29816
           4       0.50      0.53      0.51      9702
           5       0.05      0.54      0.10       821
           6       0.00      0.65      0.01        23
           7       0.02      0.86      0.04        85

    accuracy                           0.59    114748
   macro avg       0.33      0.57      0.31    114748
weighted avg       0.71      0.59      0.62    114748



In [32]:
fim_total = time.time()

In [33]:
print("Total geral do processamento (segundos): ", fim_total - ini_total)

Total geral do processamento (segundos):  1498.9269797801971


In [34]:
winsound.Beep(600, 250)
time.sleep(0.25)